In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_set_util = DataSetUtil()

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/air/air_quality_beijing_6_sites.csv', sep=',')
df = df.drop(labels=['No','day','year','month','hour','wd','station'], axis=1)
df = df.drop(labels=['No.1','day.1','year.1','month.1','hour.1','wd.1','station.1'], axis=1)
df = df.drop(labels=['No.2','day.2','year.2','month.2','hour.2','wd.2','station.2'], axis=1)
df = df.drop(labels=['No.3','day.3','year.3','month.3','hour.3','wd.3','station.3'], axis=1)
df = df.drop(labels=['No.4','day.4','year.4','month.4','hour.4','wd.4','station.4'], axis=1)
df = df.drop(labels=['No.5','day.5','year.5','month.5','hour.5','wd.5','station.5'], axis=1)
df.dropna(inplace=True)
data = data_set_util.clean_dataset(df)
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.5,SO2.5,NO2.5,CO.5,O3.5,TEMP.5,PRES.5,DEWP.5,RAIN.5,WSPM.5
23,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,28.0,14.0,20.0,1100.0,44.0,0.2,1028.6,-17.6,0.0,1.1
24,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1.4
25,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,39.0,25.0,28.0,1300.0,37.0,-0.7,1029.2,-17.9,0.0,1.0
29,9.0,11.0,19.0,70.0,500.0,20.0,-1.8,1030.1,-17.5,0.0,...,23.0,26.0,11.0,1200.0,50.0,-1.3,1028.0,-18.4,0.0,1.4
31,3.0,11.0,34.0,58.0,500.0,27.0,-1.7,1029.8,-17.0,0.0,...,6.0,19.0,11.0,1100.0,52.0,-1.3,1028.0,-17.9,0.0,0.6


In [5]:
data.describe()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,...,PM10.5,SO2.5,NO2.5,CO.5,O3.5,TEMP.5,PRES.5,DEWP.5,RAIN.5,WSPM.5
count,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,2.324800e+04,23248.000000,23248.000000,...,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000,23248.000000
mean,81.522694,109.706689,16.140662,59.594873,1285.489074,67.536062,55.341872,1.056037e+03,2.410530,0.063244,...,120.422935,14.663403,55.330626,1321.097643,77.251952,59.420518,1009.364215,1.981667,0.067004,1.384334
std,80.547645,94.143278,21.112259,37.482837,1260.612753,905.878139,1170.644511,6.631430e+03,13.609074,0.796334,...,96.820668,19.567869,36.240331,1205.038723,1294.210722,1109.252136,9.945120,13.710552,0.858220,1.175391
min,3.000000,3.000000,1.000000,2.000000,100.000000,0.214200,-9575.000000,9.877000e+02,-35.300000,0.000000,...,2.000000,1.000000,2.000000,100.000000,1.000000,-9525.000000,984.000000,-34.600000,0.000000,0.000000
25%,21.000000,37.000000,3.000000,30.000000,500.000000,8.000000,2.900000,1.004100e+03,-8.700000,0.000000,...,45.000000,2.000000,26.000000,500.000000,10.000000,3.100000,1001.200000,-9.400000,0.000000,0.600000
50%,57.000000,86.000000,8.000000,53.000000,900.000000,43.000000,13.800000,1.011800e+03,2.600000,0.000000,...,100.000000,7.000000,49.000000,900.000000,44.000000,14.000000,1008.800000,2.000000,0.000000,1.100000
75%,114.000000,156.000000,20.000000,83.000000,1600.000000,82.000000,22.900000,1.021000e+03,14.900000,0.000000,...,170.000000,19.000000,78.000000,1600.000000,83.000000,23.100000,1017.500000,14.700000,0.000000,1.800000
max,713.000000,948.000000,229.000000,290.000000,10000.000000,78183.000000,28425.000000,1.012125e+06,28.500000,46.400000,...,994.000000,500.000000,271.000000,10000.000000,99603.000000,27725.000000,1038.100000,27.400000,41.900000,12.000000


In [6]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,PM2.5(t-1),PM10(t-1),SO2(t-1),NO2(t-1),CO(t-1),O3(t-1),TEMP(t-1),PRES(t-1),DEWP(t-1),RAIN(t-1),...,PM10.5(t),SO2.5(t),NO2.5(t),CO.5(t),O3.5(t),TEMP.5(t),PRES.5(t),DEWP.5(t),RAIN.5(t),WSPM.5(t)
24,24.0,24.0,26.0,54.0,600.0,36.0,-0.2,1030.5,-17.4,0.0,...,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1.4
25,22.0,24.0,24.0,44.0,500.0,44.0,-0.4,1031.0,-17.6,0.0,...,39.0,25.0,28.0,1300.0,37.0,-0.7,1029.2,-17.9,0.0,1.0
29,14.0,17.0,21.0,36.0,400.0,50.0,-1.0,1031.3,-17.3,0.0,...,23.0,26.0,11.0,1200.0,50.0,-1.3,1028.0,-18.4,0.0,1.4
31,9.0,11.0,19.0,70.0,500.0,20.0,-1.8,1030.1,-17.5,0.0,...,6.0,19.0,11.0,1100.0,52.0,-1.3,1028.0,-17.9,0.0,0.6
32,3.0,11.0,34.0,58.0,500.0,27.0,-1.7,1029.8,-17.0,0.0,...,9.0,22.0,15.0,1200.0,51.0,-0.6,1027.7,-17.8,0.0,0.9


In [7]:
df_forecats_columns = data.loc[:,'PM2.5':'WSPM.5'].columns

In [8]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    num_fuzzy_rules = {
        "window": [],
        "qtd": []
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'PM2.5(t-1)':'WSPM.5(t-1)']
            df_test = ttest.loc[:,'PM2.5(t-1)':'WSPM.5(t-1)']
            df_original = ttest.loc[:,'PM2.5(t)':'WSPM.5(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            df_original = pd.DataFrame(df_original.values,columns=columns)
            
            #teste print fuzzy rules 
            sum_rules = 0
            for m in models:
                sum_rules = sum_rules + len(m)
                #print(len(m))
            
            #print(sum_rules)
            num_fuzzy_rules["window"].append(ct)
            num_fuzzy_rules["qtd"].append(sum_rules)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    rules = pd.DataFrame(num_fuzzy_rules)
    
    return measures,rules

## Sliding Window: MIMO KPCA-NSFTS

In [9]:
num_components_pca = 8 #10
order_fts_model = 1
npart = 65
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [10]:
kpca_result,rules =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 774
--------------------
training window 1548
--------------------
training window 2322
--------------------
training window 3096
--------------------
training window 3870
--------------------
training window 4644
--------------------
training window 5418
--------------------
training window 6192
--------------------
training window 6966


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 7740


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 8514
--------------------
training window 9288
--------------------
training window 10062
--------------------
training window 10836
--------------------
training window 11610
--------------------
training window 12384
--------------------
training window 13158
--------------------
training window 13932
--------------------
training window 14706
--------------------
training window 15480
--------------------
training window 16254
--------------------
training window 17028
--------------------
training window 17802
--------------------
training window 18576
--------------------
training window 19350
--------------------
training window 20124
--------------------
training window 20898
--------------------
training window 21672


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(t

--------------------
training window 22446


In [11]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

kpca_result.to_csv (r'all_windows_kpca_mo_ensfts_uci_air_quality_beijing_6_sites.csv', index = False, header=True)

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [12]:
print("Statistics MIMO KPCA-NSFTS(test): ")
pd.set_option('display.max_rows', None)
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,21.814,12.568,29.898,0.879,12.088,0.080,9.878,4.144,9.035,0.068,3.020,0.016
1,PM10,33.821,20.041,31.265,0.797,12.619,0.094,13.111,6.352,8.307,0.130,2.675,0.027
2,SO2,5.403,3.130,24.939,0.761,11.103,0.093,4.118,2.189,8.490,0.109,3.288,0.020
3,NO2,13.492,9.053,20.735,0.787,9.084,0.100,2.969,2.006,6.025,0.099,1.715,0.022
4,CO,331.410,203.721,21.371,0.778,9.204,0.094,164.575,103.089,7.888,0.137,2.087,0.022
5,O3,484.421,108.001,5551.162,0.724,21.719,0.119,2551.593,524.718,30011.311,0.345,9.671,0.025
6,TEMP,296.536,120.886,inf,0.602,18.471,0.114,1226.044,460.665,NaN,0.988,22.742,0.091
7,PRES,1.313,0.951,0.094,0.895,0.047,0.063,1.742,1.309,0.128,0.234,0.064,0.048
8,DEWP,1.489,0.931,inf,0.896,10.220,0.076,0.508,0.275,NaN,0.056,8.690,0.015
9,RAIN,0.513,0.083,NaN,-0.105,NaN,NaN,1.076,0.155,NaN,0.574,NaN,NaN


In [14]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_6_sites_with_std.csv', index = False, header=True)

In [15]:
#final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_air_quality_beijing_6_sites_2.csv', index = False, header=True)

In [11]:
avg_rules = round(statistics.mean(rules.loc[:,'qtd']),3)
avg_rules

467.4

## Sliding Window: MIMO PCA-NSFTS

In [12]:
num_components_pca = 20
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [13]:
pca_result,rules =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 774
--------------------
training window 1548
--------------------
training window 2322


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 3096
--------------------
training window 3870
--------------------
training window 4644
--------------------
training window 5418
--------------------
training window 6192
--------------------
training window 6966


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 7740


...

--------------------
training window 20898


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 21672


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 22446


In [18]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

pca_result.to_csv (r'all_windows_pca_mo_ensfts_uci_air_quality_beijing_6_sites.csv', index = False, header=True)

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 


print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,26.402,18.386,54.568,0.831,19.531,0.096,11.139,7.113,24.902,0.074,6.809,0.019
1,PM10,39.891,27.756,53.452,0.742,19.279,0.108,17.449,11.680,23.414,0.131,5.854,0.025
2,SO2,7.000,4.742,52.813,0.576,22.027,0.124,5.425,3.299,18.352,0.240,6.264,0.031
3,NO2,18.115,13.242,32.870,0.631,14.223,0.131,5.869,4.155,12.431,0.191,4.817,0.032
4,CO,398.570,289.403,34.894,0.705,15.345,0.110,199.718,150.988,15.092,0.151,5.614,0.022
5,O3,456.901,185.441,5101.396,0.646,31.746,0.141,2380.250,925.189,27037.144,0.291,16.922,0.033
6,TEMP,325.576,213.082,inf,0.676,19.073,0.122,1319.111,839.417,NaN,0.358,21.619,0.040
7,PRES,1.895,1.354,0.134,0.802,0.067,0.098,1.247,0.807,0.080,0.176,0.040,0.037
8,DEWP,2.233,1.592,inf,0.748,14.707,0.113,0.951,0.628,NaN,0.212,12.190,0.035
9,RAIN,0.532,0.218,inf,-0.271,97.817,NaN,1.089,0.436,NaN,0.671,3.498,NaN


In [22]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_6_sites_with_std.csv', index = False, header=True)

In [23]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_uci_air_quality_beijing_6_sites_2.csv', index = False, header=True)

In [24]:
pd.set_option('display.max_rows', None)
final_measures_pca

,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,PM2.5,26.402,18.386,54.568,0.831,19.531,0.096,11.139,7.113,24.902,0.074,6.809,0.019
1,PM10,39.891,27.756,53.452,0.742,19.279,0.108,17.449,11.680,23.414,0.131,5.854,0.025
2,SO2,7.000,4.742,52.813,0.576,22.027,0.124,5.425,3.299,18.352,0.240,6.264,0.031
3,NO2,18.115,13.242,32.870,0.631,14.223,0.131,5.869,4.155,12.431,0.191,4.817,0.032
4,CO,398.570,289.403,34.894,0.705,15.345,0.110,199.718,150.988,15.092,0.151,5.614,0.022
5,O3,456.901,185.441,5101.396,0.646,31.746,0.141,2380.250,925.189,27037.144,0.291,16.922,0.033
6,TEMP,325.576,213.082,inf,0.676,19.073,0.122,1319.111,839.417,NaN,0.358,21.619,0.040
7,PRES,1.895,1.354,0.134,0.802,0.067,0.098,1.247,0.807,0.080,0.176,0.040,0.037
8,DEWP,2.233,1.592,inf,0.748,14.707,0.113,0.951,0.628,NaN,0.212,12.190,0.035
9,RAIN,0.532,0.218,inf,-0.271,97.817,NaN,1.089,0.436,NaN,0.671,3.498,NaN


In [14]:
avg_rules = round(statistics.mean(rules.loc[:,'qtd']),3)
avg_rules

899.067